In [1]:
import requests
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
import json
from bs4 import BeautifulSoup
from pygeocoder import Geocoder

In [2]:
def mcd_scrape(url):
    response = requests.get(url)
    S = BeautifulSoup(response.text, 'lxml')
    mcd = []
    list_items = S.find_all('li')
    for L in list_items:
        if "Panera Bread in" in L.text:
            addr = L.text.replace("  ","")
            addr = addr.replace("\n","")
            addr = addr.split("Panera Bread in")
            mcd.append(addr)
    mcd2 = pd.DataFrame(mcd).drop(0,axis=1).values.tolist()

    mcd3 = []
    for elem in mcd2:
        mcd3.extend(elem)
    mcd4 = []
    for elem in mcd3:
        if elem != None:
            mcd4.append(elem)
    mcd5 = pd.DataFrame(mcd4)
    mcd6 = mcd5[0].str.split(' ',2, expand = True)
    mcd7 = mcd6[2].str.split(' ',1, expand = True).drop(0,axis=1)
    mcd8=mcd6[1].values.tolist()
    mcd9=mcd7.values.tolist()
    mcd9_list = []
    for elem in mcd9:
        mcd9_list.extend(elem)
    mcd10 = pd.DataFrame()
    mcd10['address'] = mcd9_list
    mcd10['zip'] = mcd8
    return(mcd10)

In [3]:
mo_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/panera-bread-4258/us/mo")
mo_mcd['state'] = "MO"
mo_mcd

,address,zip,state
0,1650 Clarkson Road,63017,MO
1,46 Four Seasons Shopping Center,63017,MO
2,238 Chesterfield Mall,63017,MO
3,1650 Clarkson Road,63017,MO
4,46 Four Seasons Shopping Center,63017,MO
...,...,...,...
135,147 Westport Plz,63146,MO
136,290 Mid Rivers Mall Drive,63376,MO
137,290 Mid Rivers Mall Drive,63376,MO
138,290 Mid Rivers Mall Drive,63376,MO


In [4]:
il_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/panera-bread-4258/us/il")
il_mcd['state'] = "IL"
il_mcd

,address,zip,state
0,5 W Rand Rd,60004-3132,IL
1,1734 W Algonquin Rd,60005,IL
2,31 S Evergreen Ave,60005-1427,IL
3,1736 W Algonquin Rd,60005-3405,IL
4,5 W Rand Rd,60004-3132,IL
...,...,...,...
171,3101 W White Oaks Dr,62704,IL
172,700 N Milwaukee Ave,60061,IL
173,447 N Milwaukee Ave,60061,IL
174,700 N Milwaukee Ave,60061,IL


In [5]:
both = [mo_mcd,il_mcd]
both_mcd = pd.concat(both)
cleaned_mcd = pd.DataFrame(both_mcd['zip'].str.split('-').str[0])
cleaned_mcd['address']= both_mcd['address']
cleaned_mcd['state'] = both_mcd['state']
cleaned_mcd = cleaned_mcd[['address', 'state', 'zip']]
cleaned_mcd

,address,state,zip
0,1650 Clarkson Road,MO,63017
1,46 Four Seasons Shopping Center,MO,63017
2,238 Chesterfield Mall,MO,63017
3,1650 Clarkson Road,MO,63017
4,46 Four Seasons Shopping Center,MO,63017
...,...,...,...
171,3101 W White Oaks Dr,IL,62704
172,700 N Milwaukee Ave,IL,60061
173,447 N Milwaukee Ave,IL,60061
174,700 N Milwaukee Ave,IL,60061


In [6]:
cleaned_mcd['location'] = cleaned_mcd[['address', 'state', 'zip']].agg(' '.join, axis=1)
cleaned_mcd = cleaned_mcd.drop_duplicates()
cleaned_mcd

,address,state,zip,location
0,1650 Clarkson Road,MO,63017,1650 Clarkson Road MO 63017
1,46 Four Seasons Shopping Center,MO,63017,46 Four Seasons Shopping Center MO 63017
2,238 Chesterfield Mall,MO,63017,238 Chesterfield Mall MO 63017
6,102 S 9th St,MO,65201,102 S 9th St MO 65201
7,3709 S Providence Rd,MO,65203,3709 S Providence Rd MO 65203
...,...,...,...,...
165,369 Randall Road,IL,60177,369 Randall Road IL 60177
168,3019 S Dirksen Pkwy,IL,62703,3019 S Dirksen Pkwy IL 62703
169,3101 W White Oaks Dr,IL,62704,3101 W White Oaks Dr IL 62704
172,700 N Milwaukee Ave,IL,60061,700 N Milwaukee Ave IL 60061


In [7]:
cleaned_mcd['location'].to_csv('../panera_loc.csv', index = False)
check = pd.read_csv('../panera_loc.csv')
check

,location
0,1650 Clarkson Road MO 63017
1,46 Four Seasons Shopping Center MO 63017
2,238 Chesterfield Mall MO 63017
3,102 S 9th St MO 65201
4,3709 S Providence Rd MO 65203
...,...
137,369 Randall Road IL 60177
138,3019 S Dirksen Pkwy IL 62703
139,3101 W White Oaks Dr IL 62704
140,700 N Milwaukee Ave IL 60061


In [8]:
# used https://dash.geocod.io/ to geocode data and create new csv with lat/long info
geodata = pd.read_csv('../geocoded_panera_loc.csv')
geodata

,location,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
0,1650 Clarkson Road MO 63017,38.644424,-90.564474,1.00,rooftop,1650.0,Clarkson Rd,NaN,NaN,Chesterfield,MO,St. Louis County,63017,US,St Louis
1,46 Four Seasons Shopping Center MO 63017,38.649052,-90.535804,0.22,place,NaN,NaN,NaN,NaN,Chesterfield,MO,St. Louis County,63017,US,TIGER/Line® dataset from the US Census Bureau
2,238 Chesterfield Mall MO 63017,38.651667,-90.564599,1.00,rooftop,238.0,Chesterfield Mall,NaN,NaN,Chesterfield,MO,St. Louis County,63017,US,St Louis
3,1650 Clarkson Road MO 63017,38.644424,-90.564474,1.00,rooftop,1650.0,Clarkson Rd,NaN,NaN,Chesterfield,MO,St. Louis County,63017,US,St Louis
4,46 Four Seasons Shopping Center MO 63017,38.649052,-90.535804,0.22,place,NaN,NaN,NaN,NaN,Chesterfield,MO,St. Louis County,63017,US,TIGER/Line® dataset from the US Census Bureau
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,3101 W White Oaks Dr IL 62704,39.763116,-89.709729,1.00,range_interpolation,3101.0,W White Oaks Dr,NaN,NaN,Springfield,IL,Sangamon County,62704,US,TIGER/Line® dataset from the US Census Bureau
312,700 N Milwaukee Ave IL 60061,42.237571,-87.946711,1.00,rooftop,700.0,N Milwaukee Ave,NaN,NaN,Vernon Hills,IL,Lake County,60061,US,Lake
313,447 N Milwaukee Ave IL 60061,42.234344,-87.942566,1.00,rooftop,447.0,N Milwaukee Ave,NaN,NaN,Vernon Hills,IL,Lake County,60061,US,Lake
314,700 N Milwaukee Ave IL 60061,42.237571,-87.946711,1.00,rooftop,700.0,N Milwaukee Ave,NaN,NaN,Vernon Hills,IL,Lake County,60061,US,Lake


In [9]:
#insert code here that drops all rows in cleaned_mcd that don't contain the relevant zip codes

zips = []
with open('../zip_list.txt', 'r') as f:
    zips.extend(f.read().split('\n'))
zips

['63001',
 '63005',
 '63006',
 '63011',
 '63017',
 '63021',
 '63022',
 '63024',
 '63025',
 '63026',
 '63031',
 '63032',
 '63033',
 '63034',
 '63038',
 '63040',
 '63042',
 '63043',
 '63044',
 '63045',
 '63074',
 '63088',
 '63099',
 '63105',
 '63114',
 '63117',
 '63119',
 '63121',
 '63122',
 '63123',
 '63124',
 '63125',
 '63126',
 '63127',
 '63128',
 '63129',
 '63130',
 '63131',
 '63132',
 '63133',
 '63134',
 '63135',
 '63136',
 '63137',
 '63138',
 '63140',
 '63141',
 '63143',
 '63144',
 '63145',
 '63146',
 '63151',
 '63167',
 '63198']

In [10]:
complete_mcd = geodata[~geodata.Zip.isin(zips)]
complete_mcd

,location,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
6,102 S 9th St MO 65201,38.950278,-92.327397,1.00,rooftop,102.0,S 9th St,NaN,NaN,Columbia,MO,Boone County,65201,US,Boone
7,3709 S Providence Rd MO 65203,38.909390,-92.336369,1.00,rooftop,3709.0,S Providence Rd,NaN,NaN,Columbia,MO,Boone County,65203,US,Boone
8,100 Brickton Road MO 65203,38.948232,-92.295255,0.95,rooftop,100.0,Brickton Rd,NaN,NaN,Columbia,MO,Boone County,65201,US,Boone
9,598 Columbia Dr MO 65203,38.882480,-92.397824,0.33,place,NaN,NaN,NaN,NaN,Columbia,MO,Boone County,65203,US,TIGER/Line® dataset from the US Census Bureau
10,2300 Bernadette Dr MO 65203,38.963767,-92.373859,1.00,rooftop,2300.0,Bernadette Dr,NaN,NaN,Columbia,MO,Boone County,65203,US,Boone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,3101 W White Oaks Dr IL 62704,39.763116,-89.709729,1.00,range_interpolation,3101.0,W White Oaks Dr,NaN,NaN,Springfield,IL,Sangamon County,62704,US,TIGER/Line® dataset from the US Census Bureau
312,700 N Milwaukee Ave IL 60061,42.237571,-87.946711,1.00,rooftop,700.0,N Milwaukee Ave,NaN,NaN,Vernon Hills,IL,Lake County,60061,US,Lake
313,447 N Milwaukee Ave IL 60061,42.234344,-87.942566,1.00,rooftop,447.0,N Milwaukee Ave,NaN,NaN,Vernon Hills,IL,Lake County,60061,US,Lake
314,700 N Milwaukee Ave IL 60061,42.237571,-87.946711,1.00,rooftop,700.0,N Milwaukee Ave,NaN,NaN,Vernon Hills,IL,Lake County,60061,US,Lake


In [11]:
complete_mcd = complete_mcd[["location", "Accuracy Score", "Latitude", "Longitude", "City", "State", "Zip"]]
complete_mcd = complete_mcd.rename(columns={"location":"id", "Accuracy Score":"strLocation", "Latitude":"latitude", "Longitude":"longitude", "City":"city", "State":"state", "Zip":"zip"})
complete_mcd["Store"] = "Panera Bread"
complete_mcd

,id,strLocation,latitude,longitude,city,state,zip,Store
6,102 S 9th St MO 65201,1.00,38.950278,-92.327397,Columbia,MO,65201,Panera Bread
7,3709 S Providence Rd MO 65203,1.00,38.909390,-92.336369,Columbia,MO,65203,Panera Bread
8,100 Brickton Road MO 65203,0.95,38.948232,-92.295255,Columbia,MO,65201,Panera Bread
9,598 Columbia Dr MO 65203,0.33,38.882480,-92.397824,Columbia,MO,65203,Panera Bread
10,2300 Bernadette Dr MO 65203,1.00,38.963767,-92.373859,Columbia,MO,65203,Panera Bread
...,...,...,...,...,...,...,...,...
311,3101 W White Oaks Dr IL 62704,1.00,39.763116,-89.709729,Springfield,IL,62704,Panera Bread
312,700 N Milwaukee Ave IL 60061,1.00,42.237571,-87.946711,Vernon Hills,IL,60061,Panera Bread
313,447 N Milwaukee Ave IL 60061,1.00,42.234344,-87.942566,Vernon Hills,IL,60061,Panera Bread
314,700 N Milwaukee Ave IL 60061,1.00,42.237571,-87.946711,Vernon Hills,IL,60061,Panera Bread


In [12]:
complete_mcd.to_csv('../final_panera_loc.csv', index = False)
check = pd.read_csv('../final_panera_loc.csv')
check

,id,strLocation,latitude,longitude,city,state,zip,Store
0,102 S 9th St MO 65201,1.00,38.950278,-92.327397,Columbia,MO,65201,Panera Bread
1,3709 S Providence Rd MO 65203,1.00,38.909390,-92.336369,Columbia,MO,65203,Panera Bread
2,100 Brickton Road MO 65203,0.95,38.948232,-92.295255,Columbia,MO,65201,Panera Bread
3,598 Columbia Dr MO 65203,0.33,38.882480,-92.397824,Columbia,MO,65203,Panera Bread
4,2300 Bernadette Dr MO 65203,1.00,38.963767,-92.373859,Columbia,MO,65203,Panera Bread
...,...,...,...,...,...,...,...,...
245,3101 W White Oaks Dr IL 62704,1.00,39.763116,-89.709729,Springfield,IL,62704,Panera Bread
246,700 N Milwaukee Ave IL 60061,1.00,42.237571,-87.946711,Vernon Hills,IL,60061,Panera Bread
247,447 N Milwaukee Ave IL 60061,1.00,42.234344,-87.942566,Vernon Hills,IL,60061,Panera Bread
248,700 N Milwaukee Ave IL 60061,1.00,42.237571,-87.946711,Vernon Hills,IL,60061,Panera Bread
